## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Cape Town,ZA,-33.9258,18.4232,55.90,82,0,8.01,clear sky
1,1,Busselton,AU,-33.6500,115.3333,51.26,84,0,7.67,clear sky
2,2,Tawala,PH,9.5542,123.7696,78.76,84,100,12.39,overcast clouds
3,3,Castro,BR,-24.7911,-50.0119,56.80,99,100,4.16,overcast clouds
4,4,Mataura,NZ,-46.1927,168.8643,45.48,94,100,2.10,light rain


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip?  75
What is the maximum temperature you would like for your trip?  90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,Unnamed: 0,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Tawala,PH,9.5542,123.7696,78.76,84,100,12.39,overcast clouds
10,10,Forio,IT,40.7342,13.8646,83.82,74,15,1.01,few clouds
11,11,Bethel,US,41.3712,-73.4140,76.03,93,75,9.22,mist
23,23,Albany,US,42.6001,-73.9662,76.73,88,100,1.99,overcast clouds
25,25,Faanui,PF,-16.4833,-151.7500,76.60,68,55,5.93,broken clouds
28,28,Yarada,IN,17.6500,83.2667,79.12,94,40,2.30,mist
29,29,Avarua,CK,-21.2078,-159.7750,75.25,73,75,8.05,broken clouds
30,30,Veere,NL,51.5483,3.6667,75.20,63,19,11.50,few clouds
33,33,Aktau,KZ,43.6500,51.2000,79.38,49,0,9.66,clear sky
36,36,Atuona,PF,-9.8000,-139.0333,77.68,67,68,14.88,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0             250
City                   250
Country                250
Lat                    250
Lng                    250
Max Temp               250
Humidity               250
Cloudiness             250
Wind Speed             250
Current Description    250
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tawala,PH,78.76,overcast clouds,9.5542,123.7696,
10,Forio,IT,83.82,few clouds,40.7342,13.8646,
11,Bethel,US,76.03,mist,41.3712,-73.4140,
23,Albany,US,76.73,overcast clouds,42.6001,-73.9662,
25,Faanui,PF,76.60,broken clouds,-16.4833,-151.7500,
28,Yarada,IN,79.12,mist,17.6500,83.2667,
29,Avarua,CK,75.25,broken clouds,-21.2078,-159.7750,
30,Veere,NL,75.20,few clouds,51.5483,3.6667,
33,Aktau,KZ,79.38,clear sky,43.6500,51.2000,
36,Atuona,PF,77.68,broken clouds,-9.8000,-139.0333,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")       

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name
import numpy as np
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Tawala,PH,78.76,overcast clouds,9.5542,123.7696,Panglao Regents Park Resort
10,Forio,IT,83.82,few clouds,40.7342,13.8646,Hotel Terme Providence
11,Bethel,US,76.03,mist,41.3712,-73.4140,Courtyard by Marriott Danbury
25,Faanui,PF,76.60,broken clouds,-16.4833,-151.7500,Bora-Bora Pearl Beach Resort & Spa
28,Yarada,IN,79.12,mist,17.6500,83.2667,Yarada Jungle Beach Resorts
...,...,...,...,...,...,...,...
679,Ciudad Bolivar,VE,81.18,overcast clouds,8.1222,-63.5497,Hotel Edi
685,Havelock,US,82.35,clear sky,34.8791,-76.9013,Sherwood Motel
686,Sancti Spiritus,CU,79.84,clear sky,21.9297,-79.4425,Hostal El Parque
687,Saint-Georges,GD,79.47,clear sky,12.0564,-61.7485,Radisson Grenada Beach Resort


In [37]:
# 8a. Create the output File (CSV)
output_data_file = ("../Weather_Database/WeatherPy_Vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt>
<dd>{Hotel Name}</dd>
<dt>City</dt>
<dd>{City}</dd>
<dt>Country</dt>
<dd>{Country}</dd>
<dt>Current Weather</dt>
<dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [51]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.7)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))